<a href="https://colab.research.google.com/github/charlotter62/EU-ETS-EUTL/blob/main/O2_xml_operators_byaccountID_PARSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parse Operator XML files


---


**Description**:

The following code parses operator XML files containing transaction information, downloaded from the [European Union Transaction Log](https://ec.europa.eu/clima/ets/transaction.do). The files are downloaded by accountID [xml-byregistry-bydate.ipynb](https://colab.research.google.com/drive/1lmHfv5nGsRHqT0ce6R0OiZDq_JBmTrOe?usp=sharing) script.
* The DetailsAll.xml files are downloaded with the "DetailsAll" button at the bottom of the search results.
* The TransactionsBasic.xml files are downloaded with the "Export" button.

**Author**: Charlotte Rivard
**Contact**: 15crivard@gmail.com
**Date**: 1/13/2022

*Please reach out with questions and coauthorship considerations if using this script for publications*

---



In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
!pip install lxml
from lxml import objectify
import pandas as pd
import numpy as np
import os

Combine to one XML file
(~30 minutes for accounts, 3 mins for 1000 for operators)

In [ ]:
workingdir = "/gdrive/MyDrive/Brookings/XML_downloads/xml-operators-byaccountID/"
folder = "XML files/"
files = [_ for _ in os.listdir(workingdir+folder) if _.endswith("operator.xml")]
files[0:30]

In [ ]:
len(files)

14659

In [ ]:
# [f for f in files if("83704" in f)]
# files.index("DE_83704_account.xml")

In [ ]:
startfile = files[0]
openstartfile = open(workingdir+folder+startfile, "r").read()
id = startfile.split("_")[1]
regcode = startfile.split("_")[0]
openstartfile = openstartfile.replace("<Account>\n","<Account>\n\t\t<AccountID>"+str(id)+"</AccountID>\n\t\t<RegistryCode>"+regcode+"</RegistryCode>\n")

for i in range(1,len(files)):
  nextfile = files[i]
  #Edit next file
  opennextfile = open(workingdir+folder+nextfile, "r").read()
  id = nextfile.split("_")[1]
  regcode = nextfile.split("_")[0]
  opennextfile = opennextfile.replace("<Account>\n","<Account>\n\t\t<AccountID>"+str(id)+"</AccountID>\n\t\t<RegistryCode>"+regcode+"</RegistryCode>\n")

  #Combine with full file
  openstartfile+="\n"
  openstartfile+=opennextfile
  #openstartfile = openstartfile.replace('</AccountDetails>\n<?xml version="1.0" encoding="UTF-8"?>\n<AccountDetails>\n',"")
  openstartfile = openstartfile.replace('</export>\n<?xml version="1.0" encoding="UTF-8"?>\n<export>',"")

  #Save intermittently
  if(i%500==0):
    print("Account "+str(i)+" saving xml-operators_byaccountID.xml")
    with open (workingdir+'xml-operators_byaccountID.xml', 'w') as fp:
        fp.write(openstartfile)

#Save at the end
with open (workingdir+'xml-operators_byaccountID.xml', 'w') as fp:
        fp.write(openstartfile)

Account 500 saving xml-operators_byaccountID.xml
Account 1000 saving xml-operators_byaccountID.xml
Account 1500 saving xml-operators_byaccountID.xml
Account 2000 saving xml-operators_byaccountID.xml
Account 2500 saving xml-operators_byaccountID.xml
Account 3000 saving xml-operators_byaccountID.xml
Account 3500 saving xml-operators_byaccountID.xml
Account 4000 saving xml-operators_byaccountID.xml
Account 4500 saving xml-operators_byaccountID.xml
Account 5000 saving xml-operators_byaccountID.xml
Account 5500 saving xml-operators_byaccountID.xml
Account 6000 saving xml-operators_byaccountID.xml
Account 6500 saving xml-operators_byaccountID.xml
Account 7000 saving xml-operators_byaccountID.xml
Account 7500 saving xml-operators_byaccountID.xml
Account 8000 saving xml-operators_byaccountID.xml
Account 8500 saving xml-operators_byaccountID.xml
Account 9000 saving xml-operators_byaccountID.xml
Account 9500 saving xml-operators_byaccountID.xml
Account 10000 saving xml-operators_byaccountID.xml


Parse the single XML file...(~25 min for accounts, 7 min for 1000 operators, over 12 hours for this script, need to save intermittently and re-initialize the dataframes)

In [ ]:
workingdir = "/gdrive/MyDrive/Brookings/XML_downloads/xml-operators-byaccountID/"
xml_data = objectify.parse(workingdir+'xml-operators_byaccountID.xml')
account_tags = xml_data.findall("OHADetails/Account")
account_tags

In [ ]:
len(account_tags)

In [ ]:
operatorsdf = pd.DataFrame();
installations = pd.DataFrame();
installationnames = [];
compliances = pd.DataFrame();
compliancenames = [];
relatedpersons = pd.DataFrame();
relatedpersonnames = [];
count = 0

for tag in account_tags:
  count+=1
  row = tag.getchildren()
  personflag = 0;
  acctid = "";
  regcode = "";
  operators = [];
  column_names = [];

  for item in row:
    if(item.tag == "Installation"): #If the list item has children, it is a holder installation
        installationnames = ["AccountID","RegistryCode"]+[b.tag for b in item.getchildren()]
        #print(installationnames)
        installationnames = [n for n in installationnames if n not in ["Compliance","IntCompliance"]] #Get rid of compliance for now
        #print(installationnames)
        installationrow = [acctid,regcode]+[b.text for b in item.getchildren()]
        #print(installationrow)
        installationrow = installationrow[0:len(installationnames)]
        #print(installationrow)
        installationrow = pd.DataFrame([installationrow],columns=installationnames)
        installations = pd.concat([installations,installationrow]).reset_index(drop=True) #blocks.append([blockrow])

        installationchildren = item.getchildren()
        for installchild in installationchildren:
          if(installchild.tag=="Compliance"):
            compliancenames = ["AccountID","registryCode"]+[b.tag for b in installchild.getchildren()]
            compliancerow = [acctid,regcode]+[b.text for b in installchild.getchildren()]
            compliancerow = pd.DataFrame([compliancerow],columns=compliancenames)
            compliances = pd.concat([compliances,compliancerow]).reset_index(drop=True) #blocks.append([blockrow])
    else:
      if(item.tag == "RelatedPerson"):
        if(personflag==1):
          #print("2nd related person")
          relatedpersonnames = ["AccountID","registryCode"]+[b.tag for b in item.getchildren()]
          relatedpersonrow = [acctid,regcode]+[b.text for b in item.getchildren()]
          relatedpersonrow = pd.DataFrame([relatedpersonrow],columns=relatedpersonnames)
          relatedpersons = pd.concat([relatedpersons,relatedpersonrow]).reset_index(drop=True)
        else:
          #print("1st related person")
          conum = item.find("CompanyRegistrationNo")
          if conum is not None:
            operators.append(conum.text)
            column_names.append(conum.tag)
          personflag=1
      else:
        if(item.tag not in column_names):
          operators.append(item.text)
          column_names.append(item.tag)
        if(item.tag == "AccountID"):
          acctid = item.text
        if(item.tag == "RegistryCode"):
          regcode = item.text

  operators = pd.DataFrame([operators], columns=column_names)
  operatorsdf = pd.concat([operatorsdf,operators]).reset_index(drop=True)

  if(count%500==0):
    print("Saving files at operator "+str(count))
    operatorsdf.to_csv(workingdir+"all_Operators_byAccountID2.csv",index=False,encoding="UTF-8-sig")
    relatedpersons.to_csv(workingdir+"all_RelatedPersons_byAccountID2.csv",index=False,encoding="UTF-8-sig")
    installations.to_csv(workingdir+"all_Installations_byAccountID2.csv",index=False,encoding="UTF-8-sig")
    compliances.to_csv(workingdir+"all_Compliance_byAccountID_byYear2.csv",index=False,encoding="UTF-8-sig")

In [ ]:
operatorsdf

,AccountID,RegistryCode,AccountHolderName,InstallationOrAircraftOperatorID,NationalAdministrator,NationalAdministratorCode,AccountTypeCode,AccountStatus,AccountTypeCodeLookup,CompanyRegistrationNo
0,102323,IT,VETRERIA COOPERATIVA PIEGARESE SOCIETA' COOPER...,800,Italy,IT,100,open,Holding Account,PG-56447
1,102321,IT,VEBAD S.p.A.,799,Italy,IT,100,open,Holding Account,BA-329279
2,93291,SK,"KVARTET, a.s.",172,Slovakia,SK,100,closed,Holding Account,35813954
3,97501,GB,H.J.Heinz Manufacturing UK Ltd,1002,United Kingdom,GB,100,closed,Holding Account,147624
4,103169,IT,AGRICOLA INDUSTRIALE DELLA FAELLA SPA,804,Italy,IT,100,open,Holding Account,00092830512
...,...,...,...,...,...,...,...,...,...,...
2155,104634,DE,Blue Cube Germany Assets GmbH & Co.KG,1536,Germany,DE,100,open,Holding Account,HRA 202897 Amtsgericht Tostedt
2156,103435,DE,Dow Deutschland Anlagengesellschaft mbH,4141,Germany,DE,100,open,Holding Account,HRB 200098 Amtsgericht Tostedt
2157,104145,DE,job Jenaer Objektmanagement- und Betriebs GmbH,1188,Germany,DE,100,closed,Holding Account,HRB 203302 Amtsgericht Jena
2158,104404,DE,Licher Privatbrauerei Jhring – Melchior GmbH,1371,Germany,DE,100,closed,Holding Account,HRB 6165 Amtsgericht Gießen


In [ ]:
relatedpersons

,AccountID,registryCode,Name,RelationshipTypeCode,City,Address1,LegalEntityIdentifier,RelationshipTypeCodeLookup,CountryCode,CountryCodeLookup,ZipCode,Address2,EmailAddress,Telephone1,Telephone2
0,102323,IT,VETRERIA COOPERATIVA PIEGARESE SOCIETA' COOPER...,1,PIEGARO,"VOCABOLO RINGRAZIATA PRIMA, 57",8156-0060DF45D7AB84-80,Account holder,IT,Italy,06066,NaN,NaN,NaN,NaN
1,102321,IT,VEBAD S.p.A.,1,GIOIA DEL COLLE (BA),"STRADA STATALE N. 100 - KM 35, s.n.c.",8156-003B88443C5DB2-83,Account holder,IT,Italy,70023,NaN,NaN,NaN,NaN
2,93291,SK,"KVARTET, a.s.",1,Partizánske,Nitrianska cesta 503/60,NaN,Account holder,SK,Slovakia,95801,NaN,NaN,NaN,NaN
3,97501,GB,H.J.Heinz Manufacturing UK Ltd,1,London,The Shard,NaN,Account holder,GB,United Kingdom,SE1 9SG,32 London Bridge Street,NaN,NaN,NaN
4,103169,IT,AGRICOLA INDUSTRIALE DELLA FAELLA SPA,1,REGGELLO,"E. DEL NICOLA, 29/A",NaN,Account holder,IT,Italy,50066,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,104634,DE,Blue Cube Germany Assets GmbH & Co.KG,1,Stade,"Bützflether Sand 2,",NaN,Account holder,DE,Germany,21683,NaN,NaN,NaN,NaN
2156,103435,DE,Dow Deutschland Anlagengesellschaft mbH,1,Stade,Bützflether Sand,NaN,Account holder,DE,Germany,21683,---,NaN,NaN,NaN
2157,104145,DE,job Jenaer Objektmanagement- und Betriebs GmbH,1,Jena,Rudolstädter Str. 39,NaN,Account holder,DE,Germany,07745,NaN,NaN,NaN,NaN
2158,104404,DE,Licher Privatbrauerei Jhring – Melchior GmbH,1,Lich,In den Hardtberggärten,NaN,Account holder,DE,Germany,35423,NaN,NaN,NaN,NaN


In [ ]:
installations

,AccountID,RegistryCode,PermitOrPlanDate,LastYearOfEmissions,FirstYearOfEmissions,InstallationNameOrAircraftOperatorCode,City,MainActivityTypeCode,PermitOrPlanID,Address1,MainActivityTypeCodeLookup,InstallationOrAircraftOperatorID,NationalAdministratorCode,ZipCode,ParentCompany,Address2,EPRTRIdentification,Latitude,Longitude,SubsidiaryCompany,CallSign
0,102323,IT,2006-04-13,0,2005,VETRERIA COOPERATIVA PIEGARESE,PIEGARO,7,IT-A-1201,"VOCABOLO RINGRAZIATA PRIMA, 57",Installations for the manufacture of glass inc...,800,IT,06066,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,102321,IT,2006-04-13,0,2005,VEBAD S.p.A.,GIOIA DEL COLLE (BA),7,IT-A-1200,"STRADA STATALE N. 100 - KM 35, SNC",Installations for the manufacture of glass inc...,799,IT,70023,Vetreria Cooperativa Piegarese Società Coopera...,NaN,NaN,NaN,NaN,NaN,NaN
2,93291,SK,2008-01-01,2015,2005,"Kotolňa na tuhé palivo, Parná kotolňa",Partizánske,20,305-003-2015,Nitrianska cesta 503/60,Combustion of fuels,172,SK,95801,"KVARTET, a.s., KVARTET, a.s.",NaN,NaN,NaN,NaN,NaN,NaN
3,97501,GB,2007-12-14,0,2007,H.J. Heinz – Kitt Green,Wigan,1,UK-E-IN-11757,Kitt Green,Combustion installations with a rated thermal ...,1002,GB,WN50JL,NaN,Spring Road,NaN,NaN,NaN,NaN,NaN
4,103169,IT,2006-04-04,2012,2005,AGRICOLA INDUSTRIALE DELLA FAELLA SPA,CASTELFRANCO DI SOPRA,32,IT-A-1206,"LOC PRATIGLIOLMI, 12",Manufacture of ceramics,804,IT,52020,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,104634,DE,2005-01-01,0,2005,Trichlorethylen-Anlage (Anlage H),Stade,20,14310-1013,Bützflether Sand,Combustion of fuels,1536,DE,21683,"-, -",NaN,0,NaN,NaN,"-,-",NaN
2156,103435,DE,2010-08-16,0,2010,Reservedampferzeuger,Stade,20,14310-1735,Bützflethersand,Combustion of fuels,4141,DE,21683,NaN,Dow Deutschland Anlagengesellschaft mbH,-,NaN,NaN,NaN,NaN
2157,104145,DE,2005-01-01,2019,2005,HKW Hermsdorf,Jena,20,14310-0631,Rudolstädter Str. 39,Combustion of fuels,1188,DE,07745,"-, -",NaN,0,NaN,NaN,"-,-",NaN
2158,104404,DE,2005-01-01,2016,2005,Feuerungsanlge Lich,Lich,20,14310-0826,In den Hardtberggärten,Combustion of fuels,1371,DE,35423,"-, -",NaN,0,NaN,NaN,"-,-",NaN


In [ ]:
compliances

,AccountID,registryCode,Year,ETSPhase,CumulativeVerifiedEmissions,SurrenderedAllowances,ComplianceCode,AllowanceInAllocation,UnitsSurrendered,CumulativeSurrenderedUnits,FreeAllocations,VerifiedEmissions,ReserveAllocations,TrasitionalAllocations
0,102323,IT,2005,2005-2007,72554,72554,A*,82572,72554,72554,82572,72554,NaN,NaN
1,102323,IT,2006,2005-2007,158205,85651,A,82572,85651,158205,82572,85651,NaN,NaN
2,102323,IT,2007,2005-2007,250974,NaN,B,82572,NaN,158205,82572,92769,NaN,NaN
3,102323,IT,2008,2008-2012,94821,94821,B,91096,94821,94821,91096,94821,NaN,NaN
4,102323,IT,2009,2008-2012,172611,77790,B,91096,77790,172611,91096,77790,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56155,104522,DE,2026,2021-2030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56156,104522,DE,2027,2021-2030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56157,104522,DE,2028,2021-2030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56158,104522,DE,2029,2021-2030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
operatorsdf.to_csv(workingdir+"all_Operators_byAccountID.csv",index=False,encoding="UTF-8-sig")

In [ ]:
relatedpersons.to_csv(workingdir+"all_RelatedPersons_byAccountID.csv",index=False,encoding="UTF-8-sig")

In [ ]:
installations.to_csv(workingdir+"all_Installations_byAccountID.csv",index=False,encoding="UTF-8-sig")

In [ ]:
compliances.to_csv(workingdir+"all_Compliance_byAccountID_byYear.csv",index=False,encoding="UTF-8-sig")